In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 665.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [4]:
import os
os.environ["OPENAI_API_KEY"] = '................'


In [6]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/business_cards.pdf')

In [7]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [8]:
raw_text

'Classification - Internal #\nBusiness Regalia \nTagline Soaring Ambitions. Ready For You.\nIntroA card where Business meets Luxury. \nHDFC Bank Business Regalia credit card is powered by best in class reward points \nprogram, giving you guaranteed savings on both your Business and personal spends\nCard Type Reward Points\nSelf Employed Indian : Age 21 to 65 yrs; ITR > 9 lakhs pa\nGift voucher worh Rs. 3,000 on 1 lac dpends in first 90 days\nAirport \nLounge Access12 complimentary domestic lounge access in a calendar year\n6 complimentary international lounge access per year via priority pass\nGolf Privileges-\nPriority \nCustomer \nService/ \nConceirge Yes\nDining \nExperienceSave upto 20% on dining bills paid through DineoutPAY. Maximum discount per card per \nmonth is Rs. 300 \n4 Reward Points on every Rs.150 spent \n(On all retail spends excluding wallet, emi and petrol)\n10,000 Bonus Reward Points on 5 Lakhs spends in anniversary year\nAdditional 5000 Bonus Reward Points on 8 Lakh

In [9]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [10]:
len(texts)

13

In [11]:
!pip install -U langchain-openai
#from langchain_openai import OpenAIEmbeddings

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.8 MB/s eta 0:00:00


In [12]:
from langchain_openai import OpenAIEmbeddings

In [13]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [14]:
document_search = FAISS.from_texts(texts, embeddings)

In [15]:
document_search


In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [19]:
from langchain_openai import OpenAI

In [20]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [23]:
query = "what is the welcome benefit for business regalia card"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The welcome benefit for the Business Regalia card is a gift voucher worth Rs. 3,000 on spends of Rs. 1 lakh in the first 90 days.'

In [24]:
query = "what is the welcome benefit for business moneyback card"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The welcome benefit for the Business Moneyback card is a gift voucher worth Rs. 2,000 on spending Rs. 50,000 in the first 90 days.'

In [25]:
query = "Among all cards which is having best welcome benefit"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The Business Regalia card has the best welcome benefit, with a gift voucher worth Rs. 3,000 on first 90 days spends of 1 lakh and 12 complimentary domestic lounge access in a calendar year.'